In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/invoice-ocr-data/invoice_16.jpg
/kaggle/input/invoice-ocr-data/invoice_11.jpg
/kaggle/input/invoice-ocr-data/invoice_13.jpg
/kaggle/input/invoice-ocr-data/invoice_8.jpg
/kaggle/input/invoice-ocr-data/invoice_9.jpg
/kaggle/input/invoice-ocr-data/invoice_12.jpg
/kaggle/input/invoice-ocr-data/invoice_10.jpg
/kaggle/input/invoice-ocr-data/invoice_14.jpg
/kaggle/input/invoice-ocr-data/invoice_6.jpg
/kaggle/input/invoice-ocr-data/invoice_4.jpg
/kaggle/input/invoice-ocr-data/invoice_2.jpg
/kaggle/input/invoice-ocr-data/invoice_7.jpg
/kaggle/input/invoice-ocr-data/invoice_15.jpg
/kaggle/input/invoice-ocr-data/invoice_3.jpg
/kaggle/input/invoice-ocr-data/invoice_1.jpg
/kaggle/input/invoice-ocr-data/invoice_5.jpg


In [2]:
# Generic Libraries
from PIL import Image
import os
import pandas as pd
import numpy as np
import re,string,unicodedata

#Tesseract Library
import pytesseract
from pytesseract import Output

#Warnings
import warnings
warnings.filterwarnings("ignore")

#Garbage Collection
import gc

#Gensim Library for Text Processing
import gensim.parsing.preprocessing as gsp
from gensim import utils

#TextBlob Library (Sentiment Analysis)
from textblob import TextBlob, Word

#Plotting Libraries
import matplotlib.pyplot as plt
import seaborn as sns

#WordCloud Generator
from wordcloud import WordCloud,STOPWORDS

In [3]:
# Define Directory Path
sample_images = '../input/invoice-ocr-data/'

In [4]:
# Custome Function to Traverse The Folder
def traverse(directory):
    path, dirs, files = next(os.walk(directory))
    print(path , "path")
    print(dirs , "dirs")
    print(files , "files")
    fol_nm = os.path.split(os.path.dirname(path))[-1]
    print(f'Number of files found in "{fol_nm}" : ',len(files))

In [5]:
traverse(sample_images)

../input/invoice-ocr-data/ path
[] dirs
['invoice_16.jpg', 'invoice_11.jpg', 'invoice_13.jpg', 'invoice_8.jpg', 'invoice_9.jpg', 'invoice_12.jpg', 'invoice_10.jpg', 'invoice_14.jpg', 'invoice_6.jpg', 'invoice_4.jpg', 'invoice_2.jpg', 'invoice_7.jpg', 'invoice_15.jpg', 'invoice_3.jpg', 'invoice_1.jpg', 'invoice_5.jpg'] files
Number of files found in "invoice-ocr-data" :  16


In [6]:
ex_txt = []   #list to store the extracted text

#Function to Extract Text
def TxtExtract(directory):
    """
    This function will handle the core OCR processing of images.
    """
    
    for subdir, dirs, files in os.walk(directory):
        for file in files:
            filepath = subdir + os.sep + file
            text = pytesseract.image_to_string(Image.open(filepath), timeout=5)
            if not text:
                ex_txt.extend([[file, "blank"]])
            else:   
                ex_txt.extend([[file, text]])
                
    fol_nm = os.path.split(os.path.dirname(subdir))[-1]
    
    print(f"Text Extracted from the files in '{fol_nm}' folder & saved to list..")

**Method #1 Extracting data using Data Frames**

In [7]:
# Define function to handle Extraction of Image data
def extract_Data(df,options):
    '''
    Func to Handle any data needs to be retrieved from Image
    the sent df and returns the needed values
    '''
    value = []
    final_list = []
    # selecting rows based on condition
    if len(df[df['text'].isin(options)]) != 0:
        res = df[df['text'].isin(options)]
        pos = res.index[0]
        if "Invoice" in options:
            value = [df['text'][pos:pos+20].reset_index(drop=True)]
            for i in range(len(value[0])):
                if value[0][i] == "Invoice":
                    final_list.append(value[0][i:i+5])
            final_df = pd.DataFrame(final_list)
            return final_df
        elif "Description" in options:
            value = [df['text'][pos:pos+48].reset_index(drop=True)]
            final_list.append(value[0])
            final_df = pd.DataFrame(final_list)
            return (final_df.iloc[:,:21],final_df.iloc[:,21:41],final_df.iloc[:,41:])
        else:
            return final_list
    else:
        return "OCR could not extract Such Data From Image"
        
        

In [8]:
x = pytesseract.image_to_data("../input/invoice-ocr-data/invoice_2.jpg",output_type = Output.DATAFRAME)

In [9]:
# choosing what type generic type of Data to Extract
options1 = ["Invoice"]
options2 = ['Description','DESCRIPTION']
extract_Data(x,options1)


,0,1,2,3,4,13,14,15,16,17
text,Invoice,No,:,1291908241,Vehicle,NaN,NaN,NaN,NaN,NaN
text,NaN,NaN,NaN,NaN,NaN,Invoice,Date,:,21/01/2020,Buyer


**Method #2 Extracting Data through Image to Strings output**

In [10]:
TxtExtract(sample_images)

Text Extracted from the files in 'invoice-ocr-data' folder & saved to list..


In [11]:
#Converting the list to dataframe for further analysis
ext_df = pd.DataFrame(ex_txt,columns=['FileName','Text'])
ext_df

,FileName,Text
0,invoice_16.jpg,\n\nMl Psi Li TAX INVOICE Extra Copy\n2 Daly ...
1,invoice_11.jpg,"\n\nmM Micro Plastics Prt, Ltd, TAX INVOICE T..."
2,invoice_13.jpg,"\n\nAA Mico laste Prt. Ld, TAX INVOICE Origin..."
3,invoice_8.jpg,‘ubject fo the Exclusive Jurisdiction at BANGA...
4,invoice_9.jpg,"\n\nmh Micro Plastics Prt, Ltd, TAX INVOICE O..."
5,invoice_12.jpg,"\n\n76 Mico Pht rt Ltd, TAX INVOICE Extra Cop..."
6,invoice_10.jpg,"\n\nmM Micro Plastics Prt, Ltd, TAX INVOICE D..."
7,invoice_14.jpg,"\n\n7A Mir Plats rt Ltd, TAX INVOICE Duplicat..."
8,invoice_6.jpg,\n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\...
9,invoice_4.jpg,TAX INVOICE (ORIGINAL FOR RECIPIENT)\n\nPROTEC...


In [12]:
# Create list of pre-processing func (gensim)
processes = [
               gsp.strip_tags, 
               gsp.strip_multiple_whitespaces,
               gsp.remove_stopwords, 
            ]

# Create func to pre-process text
def proc_txt(txt):
    text = utils.to_unicode(txt)
    for p in processes:
        text = p(text)
    return text

In [13]:
#Creating a new column with processed text
ext_df['Text_Pr'] = ext_df['Text'].apply(lambda x: proc_txt(x))

In [14]:
str_test = ext_df['Text_Pr'][4]
str_test

"mh Micro Plastics Prt, Ltd, TAX INVOICE Original Recipient wag “Dug Sdn MICRO PLASTICS PVT.LTD. (EOU) Page 1 1 Khata No. 170/1, Sy.No. 330/3, Subharam Indl. Estate, Marsur Gate, Kasaba Hobli, Anekal Taluk Bengaluru Karnataka 562106 India Phone Email Website : www.microplasticsindia.com GSTIN : 29AABCM9971E1Z3 CIN: U19201KA1994PTC015731 PAN AABCM9971E Invoice No: 1/EO0U192001343 Transportation Mode: Road Invoice Date & Time: 04-01-2020/15:48:16 Vehicle No. : Tax Is Payable On Reverse Charge: No Date & Time Supply: 04-01-2020 15:48:31 Payment Terms: 30 DAYS Place Supply: Karnataka Transporter Name: Eway Bill No: 191190661253 Details Recipient (Billed to) Details Consignee (Shipped to) Decathlon Sports India Pvt Ltd Decathlon Sports India Pvt Ltd SURVEY NO.78/10, CHIKKAJALA VILLAGE BELLARY ROAD, C/o DHL Supply Chain India Pvt Ltd, Survey 41/2 Oblapura Village BANGALORE Anugondanahalli Hoskote Taluk Bengaluru Bengaluru 562157 Karnataka India Bengaluru 560067 Karnataka India State Code : 2

In [15]:
import re
def extract_data_From_String(data,value,window):
    ''' Func To handle Data Extracting Based on the Choices
        of paramters passed such as window size refers to how long 
        the text should be detected and value which refers to which
        piece of Info needed to be Extracted from the Image
    '''
    ext_data = []
    if value == "Invoice":
        for i in range(len(data)):
            ext_pos  = data[i].find(value.capitalize())
            output = data[i][ext_pos:ext_pos + window]
            if len(output) != 0:
                try:
                    Invoice_No_Match   = re.search(r'[{1}]/\b[A-Z0-9]{10,15}|\d{9,10}',output)
                    Invoice_Date_Match = re.search(r'([\d]{2})-([\d]{2})-([\d]{4})|([\d]{2}).([\d]{2}).([\d]{4})',output)
                    print("Image No",i+1)
                    print("Invoice No.", Invoice_No_Match.group())
                    print("Invoice Date ",Invoice_Date_Match.group())
                    print("\n")
                except:
                    print("No Match")
            else:
                return ext_data
    if value == "Description":
        for i in range(len(data)):
            ext_pos  = data[i].find(value.capitalize())
            output = data[i][ext_pos:ext_pos + window]
            if len(output) != 0:
                try:
                    print("\n")
                    Item_QTY_Match   = re.search(r'[\d]{3,6}[\]}|*-] [A-z]{3}|[\d]{3,6}[\]}|*-] [a-z]{3}|[\d]{3,6}[\]}|*-] [A-Z]{3}',output)
                    Item_Amount_Match = re.search(r'[\d]{4,5}[.,][\d]{2}[\]}|*-]',output)
                    Item_Desc_Match = re.search(r'[A-z]{4,10} [a-z]{2,10} [\d]{1,5}|[A-Z]{4,10} [A-Z]{2,10} [\d]{1,5}',output)
                    print("Image No",i+1)
                    print("Item QTY ", Item_QTY_Match.group())
                    print("Item Amount ",Item_Amount_Match.group())
                    print("Item Description ",Item_Desc_Match.group())
                    print("\n")
                except:
                    pass
            else:
                ext_pos  = data[i].find(value.upper())
                output = data[i][ext_pos:ext_pos + window]
                if len(output) != 0:
                    try:
                        print("\n")
                        Item_QTY_Match   = re.search(r'[\d]{3,6}[\]}|*-] [A-z]{3}|[\d]{3,6}[\]}|*-] [a-z]{3}|[\d]{3,6}[\]}|*-] [A-Z]{3}',output)
                        Item_Amount_Match = re.search(r'[\d]{4,5}[.,][\d]{2}[\]}|*-]',output)
                        Item_Desc_Match = re.search(r'[A-z]{4,10} [a-z]{2,10} [\d]{1,5}|[A-Z]{4,10} [A-Z]{2,10} [\d]{1,5}',output)
                        print("Image No",i+1)
                        print("Item QTY ", Item_QTY_Match.group())
                        print("Item Amount ",Item_Amount_Match.group())
                        print("Item Description ",Item_Desc_Match.group())
                        print("\n")
                    except:
                        pass
                else:
                    return ext_data
        
        return ext_data

**Testing The Model**

## Item Line Description

In [16]:
extract_data_From_String(ext_df['Text_Pr'],'Description', 600)



Image No 1
Item QTY  10080] Nos
Item Amount  24676.80|
Item Description  Jump rope 100




Image No 2
Item QTY  360} NOS
Item Amount  19890.00]
Item Description  MASSAGE PURPLE 95030




Image No 3
Item QTY  10080] Nos
Item Amount  24676.80|
Item Description  Jump rope 100




Image No 4


Image No 5
Item QTY  360} NOS
Item Amount  19890.00]
Item Description  MASSAGE PURPLE 95030




Image No 6
Item QTY  360} NOS
Item Amount  19890.00]
Item Description  MASSAGE PURPLE 95030




Image No 7
Item QTY  360} NOS
Item Amount  19890.00]
Item Description  MASSAGE PURPLE 95030




Image No 8
Item QTY  10080] Nos
Item Amount  24676.80|
Item Description  Jump rope 100




[]

## Inoice Number and Invoice Date

In [17]:
extract_data_From_String(ext_df['Text_Pr'],'Invoice', 100)

Image No 1
Invoice No. 1/EQU192001344
Invoice Date  04-01-2020


Image No 2
Invoice No. 1/EO0U192001343
Invoice Date  04-01-2020


Image No 3
Invoice No. 1/EQU192001344
Invoice Date  04-01-2020


Image No 4
Invoice No. 2190002016
Invoice Date  2190002016


Image No 5
Invoice No. 1/EO0U192001343
Invoice Date  04-01-2020


Image No 6
Invoice No. 1/EO0U192001343
Invoice Date  04-01-2020


Image No 7
Invoice No. 1/EO0U192001343
Invoice Date  04-01-2020


Image No 8
Invoice No. 1/EQU192001344
Invoice Date  04-01-2020




[]